In [1]:
from mlscorecheck.aggregated import (Fold, Dataset, Experiment)
from mlscorecheck.aggregated import solve

In [2]:
experiment = Experiment(datasets=[{'name': 'common_datasets.ADA', 'aggregation': 'mor'},
                                    {'name': 'common_datasets.ecoli1', 'n_folds': 5, 'n_repeats': 3, 'aggregation': 'mor'}],
                        aggregation='mor')

2023-08-24 00:26:13,134:INFO:querying p and n from looking up the dataset
2023-08-24 00:26:13,137:INFO:creating a folding based on the specification
2023-08-24 00:26:13,138:INFO:querying p and n from looking up the dataset
2023-08-24 00:26:13,139:INFO:creating a folding based on the specification


In [3]:
experiment.sample()

In [4]:
scores = experiment.calculate_scores()

In [5]:
scores

{'acc': 0.39809258964824945,
 'sens': 0.7193808713961776,
 'spec': 0.2903391117470656,
 'bacc': 0.5048599915716216}

In [6]:
res = solve(experiment, scores, {'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

{'acc': 0.39809258964824945, 'sens': 0.7193808713961776, 'spec': 0.2903391117470656, 'bacc': 0.5048599915716216}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': 1*tp_ADA_0_0 + 1*tp_ecoli1_0_0 + 1*tp_ecoli1_0_1 + 1*tp_ecoli1_0_2 + 1*tp_ecoli1_0_3 + 1*tp_ecoli1_0_4 + 1*tp_ecoli1_1_0 + 1*tp_ecoli1_1_1 + 1*tp_ecoli1_1_2 + 1*tp_ecoli1_1_3 + 1*tp_ecoli1_1_4 + 1*tp_ecoli1_2_0 + 1*tp_ecoli1_2_1 + 1*tp_ecoli1_2_2 + 1*tp_ecoli1_2_3 + 1*tp_ecoli1_2_4 + 0, 'tn': 1*tn_ADA_0_0 + 1*tn_ecoli1_0_0 + 1*tn_ecoli1_0_1 + 1*tn_ecoli1_0_2 + 1*tn_ecoli1_0_3 + 1*tn_ecoli1_0_4 + 1*tn_ecoli1_1_0 + 1*tn_ecoli1_1_1 + 1*tn_ecoli1_1_2 + 1*tn_ecoli1_1_3 + 1*tn_ecoli1_1_4 + 1*tn_ecoli1_2_0 + 1*tn_ecoli1_2_1 + 1*tn_ecoli1_2_2 + 1*tn_ecoli1_2_3 + 1*tn_ecoli1_2_4 + 0, 'p': 1260, 'n': 3895, 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.0004901960784313725*tn_ecoli1_0_0 + 0.0004975124378109452*tn_ecoli1_0_1 + 0.0004975124378109452*tn_ecoli1_0_2 + 0.0004975124378109452*tn_ecoli1_0_3 + 0.000497512437810

In [7]:
experiment2 = Experiment(**experiment.to_dict(raw_problem=True))

In [8]:
experiment2.populate_with_solution(res)

In [9]:
experiment2.calculate_scores()

{'acc': 0.39800535613256993,
 'sens': 0.7194298671849693,
 'spec': 0.2903846153846154,
 'bacc': 0.5049072412847924}

In [10]:
experiment2.check_bounds()

{'figures': {'tp': 922.0, 'tn': 453.0, 'p': 1260, 'n': 3895},
 'scores': {'acc': 0.39800535613256993,
  'sens': 0.7194298671849693,
  'spec': 0.2903846153846154,
  'bacc': 0.5049072412847924},
 'downstream_bounds': True,
 'datasets': [{'figures': {'tp': 758.0, 'tn': 0.0, 'p': 1029, 'n': 3118},
   'scores': {'acc': 0.18278273450687244,
    'sens': 0.7366375121477162,
    'spec': 0.0,
    'bacc': 0.3683187560738581},
   'score_bounds': None,
   'check_score_bounds': True,
   'tptn_bounds': None,
   'check_tptn_bounds': True,
   'downstream_bounds': True,
   'folds': [{'figures': {'tn': 0.0, 'tp': 758.0},
     'scores': {'acc': 0.18278273450687244,
      'sens': 0.7366375121477162,
      'spec': 0.0,
      'npv': 0.0,
      'ppv': 0.195562435500516,
      'bacc': 0.3683187560738581,
      'f1p': 0.309072375127421,
      'fm': 0.379550557839991,
      'p': 1029,
      'n': 3118,
      'tn': 0.0,
      'tp': 758.0},
     'score_bounds': None,
     'check_score_bounds': True,
     'tptn_boun

In [11]:
#import pulp as pl
#lp_program = pl.LpProblem('feasibility')

In [12]:
#tp = pl.LpVariable('tp', 0, 5, pl.LpInteger)
#tn = pl.LpVariable('tn', 0, 10, pl.LpInteger)

In [13]:
#lp_program += tp

In [14]:
#lp_program

In [15]:
#lp_program += tp <= 5

In [16]:
#lp_program

In [17]:
dataset = Dataset(name='common_datasets.ADA', n_repeats=2, n_folds=3, aggregation='rom')

2023-08-24 00:26:13,357:INFO:querying p and n from looking up the dataset
2023-08-24 00:26:13,358:INFO:creating a folding based on the specification


In [18]:
dataset.simulate()

AttributeError: 'Dataset' object has no attribute 'simulate'

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2 = Dataset(**dataset.to_dict())

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2.aggregation = 'mor'

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.scores

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.linear_programming

In [ ]:
res = solve(dataset, dataset2.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

aaa
bbb
{'acc': 0.4115200984736789, 'sens': 0.3542274052478134, 'spec': 0.43045704696330295, 'bacc': 0.3923422261055582}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0, 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0, 'p': 2058, 'n': 6236, 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0, 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 

In [ ]:
dataset2.populate_with_solution(res)

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4114809454896286,
 'sens': 0.3542274052478134,
 'spec': 0.43038177735001604,
 'bacc': 0.3923045912989147}

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset.linear_programming

{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0,
 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0,
 'p': 2058,
 'n': 6236,
 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0,
 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 + 0.00048590864917395527*tp_ADA_1_1 + 0.00048590864917395527*tp_ADA_1_2 + 0.0,
 'spec': 0.00016035920461834508*tn_ADA_0_0 + 0.00016035920461834508*tn_ADA_0_1 + 0.000160359204618345

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
fold = Fold(p=5, n=10, id='fold_0')

In [ ]:
fold2 = Fold(**fold.to_dict(True))

In [ ]:
fold2.to_dict()

{'p': 5,
 'n': 10,
 'id': 'fold_0',
 'score_bounds': None,
 'tptn_bounds': None,
 'evaluation': None,
 'scores': None}

In [ ]:
fold.simulate()

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.evaluation

{'tp': 0.0, 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.p, fold.n

(5, 10)

In [ ]:
res = solve(fold, fold.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

{'acc': 0.13333333333333333, 'sens': 0.0, 'spec': 0.2, 'npv': 0.2857142857142857, 'ppv': 0.0, 'bacc': 0.1, 'f1p': 0.0, 'fm': 0.0, 'p': 5, 'n': 10, 'tp': 0.0, 'tn': 2.0}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': tp_fold_0, 'tn': tn_fold_0, 'acc': 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0 + 0.0, 'sens': 0.2*tp_fold_0 + 0.0, 'spec': 0.1*tn_fold_0 + 0.0, 'bacc': 0.05*tn_fold_0 + 0.1*tp_fold_0 + 0.0}
0.13333333333333333 0.0001 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0
0.0 0.0001 0.2*tp_fold_0
0.2 0.0001 0.1*tn_fold_0
0.1 0.0001 0.05*tn_fold_0 + 0.1*tp_fold_0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.sol (default strategy 1)
At line 2

In [ ]:
res.status

1

In [ ]:
res.variables()

[tn_fold_0, tp_fold_0]

In [ ]:
fold.populate_with_solution(res).calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}

In [ ]:
fold.evaluation

{'tn': 2.0, 'tp': 0.0}

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}